In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

## Define where the images are located and what data to consider

In [ ]:
folder_path = "./cellpose_train_imgs/24-03-08_pred01"

In [ ]:
mode = "all"
bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:5]

## Create predictions/segmentations on the cellpose dataset (with self-created scribbles)

In [ ]:
# Define the convpaint parameters
layer_list = [0] # layers to use for convpaint (depending on the model chosen)
scalings = [1, 2] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = None # seed used for random forest classifier

# Predictions
for img_num in range(0, 10):
    for bin in bins:
        for s in suff:
            print(f"IMG {img_num}: {bin}_{s}")
            pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=layer_list, scalings=scalings, model=model, random_state=random_state, save_res=True, show_res=False)
            pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, suff=s, save_res=True, show_res=False)

Same cell but for testing...

In [ ]:
folder_path = "./results/data/pred04_examples"
# folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.1]#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]

# Define the convpaint parameters
layer_list = [0] # layers to use for convpaint (depending on the model chosen)
scalings = [1, 2] # scalings for convpaint (downscaling the image)
model="single_layer_vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = 123 # seed used for random forest classifier

# Predictions
for img_num in [0]:#(0,189):#range(0, 10):
    for bin in [0.01]: #bins:
        for s in ["a"]:#suff:
            print(f"IMG {img_num}: {bin}_{s}")
            # get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=False, load_scribbles=False, mode=mode, bin=bin, suff=s, load_pred=False, pred_tag="convpaint")
            # pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=layer_list, scalings=scalings, model=model, random_state=random_state, save_res=False, show_res=True)
            # pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, suff=s, save_res=False, show_res=True)

Testing VGG16 options

In [ ]:
folder_path = "./results/data/pred04_examples"
# folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.1]#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]
s = "a"

# Define the convpaint parameters
layer_list = [[0], [0,2], [0,2,5]] # layers to use for convpaint (depending on the model chosen)
scalings = [[1]]#, [1,2], [1,2,4], [1,2,4,8]] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = 123 # seed used for random forest classifier

# Predictions
for img_num in [398]:#range(0, 540, 10):#range(0, 10):
    for bin in [0.1]: #bins:
        for sc in [[1,2]]:#scalings:
            for l in [[0,2,5]]:#layer_list:
                print(f"IMG {img_num}: bin {bin}, scalings {sc}, layers {l}")
                pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=l, scalings=sc, model=model, random_state=random_state, save_res=False, show_res=True)
                # get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=False, load_scribbles=False, mode=mode, bin=bin, suff=s, load_pred=False, pred_tag="convpaint")
                # pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=layer_list, scalings=scalings, model=model, random_state=random_state, save_res=False, show_res=True)
                # pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, suff=s, save_res=False, show_res=True)

In [ ]:
labels = np.array(Image.open('D:/Documents/Beruf & Ausbildung/Ausbildung/22-08 MSC Bioinf/24FS_Master Thesis/02_scribble_datasets_tests/cellpose/results/data/pred04_examples/398_scribbles_all_00100_a.png'))
image = np.array(Image.open('D:/Documents/Beruf & Ausbildung/Ausbildung/22-08 MSC Bioinf/24FS_Master Thesis/02_scribble_datasets_tests/cellpose/results/data/pred04_examples/398_img.png'))
pred = selfpred_convpaint(image, labels, layer_list=[0,2,5], scalings=[1,2], model="vgg16", random_state=123)

In [20]:
from napari_convpaint.conv_paint_utils import (Hookmodel, filter_image_multioutputs, get_features_current_layers, get_multiscale_features, train_classifier, predict_image, train_test_split)
from sklearn.ensemble import RandomForestClassifier
layer_list=[0,2,5]
scalings=[1,2]
model="vgg16"
random_state=123

if len(image.shape) == 3 and image.shape[2] < 4:
    image = np.moveaxis(image, 2, 0) # Convpaint expects (C, H, W)
# Define the model
model = Hookmodel(model_name=model)
# Ensure the layers are given as a list


# if isinstance(layer_list, int):
#     layer_list = [layer_list]
# # Read out the layer names
# all_layers = [key for key in model.module_dict.keys()]
# layers = [all_layers[i] for i in layer_list]
layers = ['features.0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))',
          'features.2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))',
          'features.5 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))']



# Register the hooks for the selected layers
model.register_hooks(selected_layers=layers)
# Get the features and targets
features, targets = get_features_current_layers(
    model=model, image=image, annotations=labels, scalings=scalings,
    order=1, use_min_features=False, image_downsample=1)

# Do RF training manually (copied from convpaint utils) to have access to seed/random_state:
X, X_test, y, y_test = train_test_split(features, targets,
                                        test_size=0.2,
                                        random_state=42)
random_forest = RandomForestClassifier(n_estimators=100, random_state = random_state)
random_forest.fit(X, y)

# Predict on the image
predicted = predict_image(
    image, model, random_forest, scalings=scalings,
    order=1, use_min_features=False, image_downsample=1)

RuntimeError: Given input size: (64x1x4). Calculated output size: (64x0x2). Output size is too small

Some notes about the model usage of convpaint:

In [ ]:

### VGG16 model choices: 'vgg16', 'single_layer_vgg16', 'single_layer_vgg16_rgb' (others: 'efficient_netb0', 'dino_vits16')
# if model_name == 'vgg16':
#     self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
#     # self.transform =  models.VGG16_Weights.IMAGENET1K_V1.transforms()
# elif model_name == 'efficient_netb0':
#     self.model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
# elif model_name == 'single_layer_vgg16':
#     self.model = load_single_layer_vgg16(keep_rgb=False)
# elif model_name == 'single_layer_vgg16_rgb':
#     self.model = load_single_layer_vgg16(keep_rgb=True)
# elif model_name == 'dino_vits16':
#     self.model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

# TEST ILASTIK

In [ ]:
folder_path = "./cellpose_train_imgs/24-03-08_pred01"
img = np.array(Image.open(f"{folder_path}/000_img.png")) # NOTE: we need to take only one channel for Ilastik to work (so far)
print(len(img.shape))
# img = np.moveaxis(img, -1, 0)
# print(img.shape)
labels = np.array(Image.open(f"{folder_path}/000_scribbles_all_00100_a.png"))
print(len(labels.shape))

In [ ]:
v = napari.Viewer()
v.add_image(img, name="img")
v.add_labels(labels, name="labels")

In [ ]:
pred = pixel_classification_ilastik(img, labels)

In [ ]:
v.add_labels(pred, name="Ilastik")

In [ ]:
v.add_labels(np.array(Image.open(f"{folder_path}/000_convpaintCh1_all_0.1_a.png")), name="Convpaint")